In [2]:
import wave
import numpy as np

# read file 
FPB = 2048
file_path = "example.wav"
# 读取本地文件
with wave.open(file_path, 'rb') as wf:
    signal = wf.readframes(FPB)
    sound = np.frombuffer(signal, dtype=np.int16).astype(np.float32) / 32768

print("sound shape: ", sound.shape)

Read success
sound shape:  (2048,)


In [4]:
import soundfile as sf
import rir_generator as rir
import scipy.signal as ss
import numpy as np
import wave

def add_room_reverb(audio_path, save_path, 
                   room_dim=[5, 4, 6],
                   source_pos=[2, 3.5, 2],
                   receiver_pos=[2, 1.5, 2],
                   rt60=0.4):
    """
    添加基于房间模型的混响效果
    
    参数:
    - audio_path: 输入音频文件路径
    - save_path: 输出音频文件路径
    - room_dim: 房间尺寸 [x y z] (米)
    - source_pos: 声源位置 [x y z] (米)
    - receiver_pos: 接收器位置 [x y z] (米)
    - rt60: 混响时间(秒)
    """
    # 使用 wave 读取音频
    with wave.open(audio_path, 'rb') as wav:
        # 获取音频参数
        fs = wav.getframerate()
        n_channels = wav.getnchannels()
        sampwidth = wav.getsampwidth()
        n_frames = wav.getnframes()
        
        # 读取音频数据并转换为numpy数组
        signal = wav.readframes(n_frames)
        signal = np.frombuffer(signal, dtype=np.int16)
        
        # 重塑数组为(n_samples, n_channels)
        signal = signal.reshape(-1, n_channels)
        # 转换为float类型进行处理
        signal = signal.astype(np.float32) / 32768.0

    # 生成房间冲击响应
    h = rir.generate(
        c=340,                      # 声速 (m/s)
        fs=fs,                      # 采样率
        r=[receiver_pos],           # 接收器位置
        s=source_pos,               # 声源位置
        L=room_dim,                 # 房间尺寸
        reverberation_time=rt60,    # 混响时间
        nsample=4096                # 输出样本数
    )
    
    # 确保h形状正确 (nsample, 1)
    h = h.reshape(-1, 1)
    
    # 对每个声道进行卷积
    reverbed = np.zeros_like(signal)
    for i in range(signal.shape[1]):
        reverbed[:, i] = ss.convolve(signal[:, i], h[:, 0], mode='same')
    
    # 归一化输出
    reverbed = reverbed / np.max(np.abs(reverbed))
    
    # 保存处理后的音频
    sf.write(save_path, reverbed, fs)
    
    return reverbed, fs


add_room_reverb("example.wav", "example_reverbed.wav")

(array([[-2.8061587e-04],
        [-1.7931614e-03],
        [-2.2212041e-03],
        ...,
        [-5.8562870e-05],
        [-4.4033848e-05],
        [-4.7326383e-05]], dtype=float32),
 8000)

In [6]:
import pyaudio

def add_room_reverb(input_signal, fs,
                   room_dim=[5.0, 4.0, 6.0],
                   source_pos=[2.0, 3.5, 2.0],
                   receiver_pos=[2.0, 1.5, 2.0],
                   rt60=0.4):
    h = rir.generate(
        c=340,
        fs=fs,
        r=[receiver_pos],
        s=source_pos,
        L=room_dim,
        reverberation_time=rt60,
        nsample=4096
    )
    h = h.reshape(-1, 1)
    reverbed = ss.convolve(input_signal, h[:, 0], mode='same')
    max_val = np.max(np.abs(reverbed))
    if max_val > 1e-6:
        reverbed = reverbed / max_val
    else:
        reverbed = np.zeros_like(reverbed)
    return reverbed


audio_path = "example.wav"
with wave.open(audio_path, 'rb') as wav:
    # 获取音频参数
    fs = wav.getframerate()
    n_channels = wav.getnchannels()
    sampwidth = wav.getsampwidth()
    n_frames = wav.getnframes()
    
    # 读取音频数据并转换为numpy数组
    signal = wav.readframes(n_frames)
    signal = np.frombuffer(signal, dtype=np.int16)
    
    # 重塑数组为(n_samples, n_channels)
    signal = signal.reshape(-1, n_channels)
    # 转换为float类型进行处理
    signal = signal.astype(np.float32) / 32768.0

reverbed = add_room_reverb(signal[:, 0], fs)


# 准备音频播放参数
CHANNELS = 1
RATE = fs
WIDTH = 2  # 16位采样
FPB = 1024  # 每个缓冲区的帧数
# 将float32转换回int16进行播放
audio_data = (reverbed * 32767).astype(np.int16)

# 初始化PyAudio
p = pyaudio.PyAudio()

# 打开音频流
stream = p.open(
    format=p.get_format_from_width(WIDTH),
    channels=CHANNELS,
    rate=RATE,
    input=False,
    output=True,
    frames_per_buffer=FPB
)

# 播放音频
try:
    stream.write(audio_data.tobytes())
finally:
    # 关闭音频流
    stream.stop_stream()
    stream.close()
    p.terminate()